# Action2

一个农民承包了6块耕地共300亩，准备播种小麦，玉米，水果和蔬菜四种农产品，已知各种农产品的计划播种面积、每块土地种植不同农产品的单产收益     如何进行合理安排，使得总收益最大     可以使用pulp/ortools工具

In [47]:
import pulp
import numpy as np
from pprint import pprint

#定义问题
def farming_problem(costs, x_max, y_max):
    row = len(costs)
    col = len(costs[0])
    print(row)
    print(col)
    # 定义问题
    prob = pulp.LpProblem('Farming', sense=pulp.LpMaximize)
    # 定义决策变量
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
    #递归展开列表
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x] 
    print('flatten=',flatten)
    print(var)
    
    #定义目标函数
    prob += pulp.lpDot(flatten(var), costs.flatten())
    
    #定义约束条件
    for i in range(row):
        prob += pulp.lpSum(var[i]) <= x_max[i]
    for j in range(col):
        prob += pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j]
#         prob += pulp.lpSum(var[i][j]) <= y_max[i]
    prob.solve()
    
    result = {'object':pulp.value(prob.objective),
              'var':[[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}
    return result


costs = np.array([[500, 550, 630, 1000, 800, 700],
                  [800, 700, 600, 950, 900, 930],
                  [1000, 960, 840, 650, 600, 700],
                  [1200, 1040, 980, 860, 880, 780]])
#最大产量
max_plant = [76,88,96,40]
#最大耕种
max_cultivation = [42,56,44,39,60,59]
#使用线性规划
result = farming_problem(costs, max_plant, max_cultivation)

4
6
flatten= <function farming_problem.<locals>.<lambda> at 0x000002724CA8BE50>
[[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]


In [48]:
result

{'object': 284230.0,
 'var': [[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
  [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
  [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]}

In [49]:
print(f'最大值为{result["object"]}')
print('各变量的取值为：')
pprint(result['var'])

最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]


第一块第4个土地小麦种了39个